In [ ]:
# add dependencies
import pandas as pd
import os
import datetime
import matplotlib.pyplot as plt
import numpy as np
import time

from sqlalchemy import create_engine
from sqlalchemy.orm import Session
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()
from sqlalchemy.ext.automap import automap_base

from sqlalchemy import Column, Integer, String, Float, and_, Date, desc, func

In [ ]:
# Use create the engine and assign classes
engine = create_engine("sqlite:///hawaii.sqlite")

# Declare a Base using `automap_base()`
Base = automap_base()

# Use the Base class to reflect the database tables
Base.prepare(engine, reflect=True)

Station = Base.classes.Stations
Measurement = Base.classes.Measurements

session = Session(engine)